In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from stempel import StempelStemmer
import re

In [21]:
dataset = pd.read_csv('criminal.csv')

In [22]:
def transform_text(text):
    text = text.lower()
    text = text.split()
    ss = StempelStemmer.default()
    text = [ss.stem(word) for word in text]
    if None in text:
        text.remove(None)
    text = ' '.join(text)
    return text

In [23]:
corpus = []
for i in range(dataset.shape[0]):
    art = transform_text(dataset['crime'][i])
    corpus.append(art)

In [40]:
transformed_criminal = pd.DataFrame(corpus, columns=['crime'])
transformed_criminal

,crime
0,§ 1. kto wszczyna lub prowadzić wojnę napastniczy
1,§ 3. kto publiczny nawoływać do wszcząć wojny ...
2,"§ 1. kto, w ć wyniszczeć w całość alć w część ..."
3,"§ 2. kto, w ć wyniszczeć w całość alć w część ..."
4,"§ 1. kto, brać udzieć w masowy zamach lub choć..."
...,...
69,kto nieumyślny powodować śmierć człowieić
70,§ 1. kto powodować ciężki uszczerbka na zdrowi...
71,§ 1. kto naywać inny osoba do spowodować u on ...
72,§ 2. kto przemoca lub groźba bezprawny zmuszać...


In [41]:
for i in range(transformed_criminal.shape[0]):
    transformed_criminal['crime'][i] = re.sub('[^a-zA-ZąĄćĆęĘłŁńŃóÓśŚźŹżŻ]', ' ', transformed_criminal['crime'][i])
    transformed_criminal['crime'][i] = transformed_criminal['crime'][i].strip()

In [42]:
transformed_criminal.to_csv('transformed.csv', sep=';', encoding='utf-8-sig')

In [43]:
def fit_crime(my_crime, data):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)
    crime_vector = tfidf_vectorizer.transform([my_crime])
    similarity = cosine_similarity(crime_vector, tfidf_matrix)
    print(similarity)
    best_match_index = similarity.argsort()[0][-1]
    return dataset.iloc[best_match_index]['article_number'], dataset.iloc[best_match_index]['crime'], dataset.iloc[best_match_index]['penalty']

In [47]:
wykroczenie_input = "Zabiłem kobietę"
wykroczenie_input = transform_text(wykroczenie_input)
print(wykroczenie_input)
numer, opis, kara = fit_crime(wykroczenie_input, corpus)

print(f"Dopasowane przestępstwo: {opis}")
print(f"Grożąca kara: {kara}")

zabić kobieta
[[0.         0.         0.         0.         0.         0.06370539
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.32658897 0.22624769 0.38025562
  0.35514193 0.3902403  0.41785418 0.         0.         0.
  0.         0.        ]]
Dopasowane przestępstwo: § 1. Jeżeli następstwem czynu: Kto za zgodą kobiety przerywa jej ciążę z naruszeniem przepisów
ustawy lub kto udziela kobiecie ciężarnej pomocy
w przerwaniu ciąży z naruszeniem przepisów ustawy lub j